In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [51]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [33]:
df = pd.DataFrame()
df = pd.read_csv("../input/sms-spam-collection-dataset/spam.csv",encoding='latin1')
df.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [34]:
#df = df.head(3)
df.shape

(5572, 5)

In [35]:
df.v1 = LabelEncoder().fit_transform(df.v1)


In [36]:
review_lines = list()
lines = df['v2'].values.tolist()

for line in lines:   
    tokens = word_tokenize(line)
    
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    
    # remove punctuation from each word    
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    
    # filter out stop words    
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    review_lines.append(words)

In [38]:
len(review_lines)


5572

In [39]:
EMBEDDING_DIM = 100
# train word2vec model
model = gensim.models.Word2Vec(sentences=review_lines, size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
#model = gensim.models.Word2Vec(sentences=review_lines)
# vocab size
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 7782


In [14]:
#words

In [40]:
# save model in ASCII (word2vec) format
filename = 'spam_ham_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)
#/kaggle/working/spam_ham_embedding_word2vec.txt


In [41]:
model.wv

In [43]:
model.wv.most_similar('hard')#, topn =1)

[('life', 0.9970679879188538),
 ('much', 0.9970424175262451),
 ('wat', 0.9970350861549377),
 ('went', 0.9970228672027588),
 ('amp', 0.9969656467437744),
 ('sure', 0.996953010559082),
 ('job', 0.9969454407691956),
 ('getting', 0.996932864189148),
 ('today', 0.9969325065612793),
 ('n', 0.9969313740730286)]

In [47]:
#Let’s see the result of semantically reasonable word vectors (king - man + woman)
model.wv.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])

#odd word out
print(model.wv.doesnt_match("woman king queen movie".split()))

queen


In [48]:

import os

embeddings_index = {}
f = open(os.path.join('', 'spam_ham_embedding_word2vec.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

In [49]:
df.shape

(5572, 5)

In [23]:
# X_train = df.loc[:2, 'v2'].values
# y_train = df.loc[:2, 'v1'].values
# X_test = df.loc[2:, 'v2'].values
# y_test = df.loc[2:, 'v1'].values

In [57]:
X_train, X_test, y_train, y_test = train_test_split(df['v2'], df['v1'], test_size=0.2, random_state = 7)

In [58]:
X_train.shape,X_test.shape

((4457,), (1115,))

In [59]:
total_reviews = X_train + X_test
max_length = 100 # try other options like mean of sentence lengths

In [60]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

VALIDATION_SPLIT = 0.2

# vectorize the text samples into a 2D integer tensor
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(review_lines)
sequences = tokenizer_obj.texts_to_sequences(review_lines)

# pad sequences
word_index = tokenizer_obj.word_index
print('Found %s unique tokens.' % len(word_index))

review_pad = pad_sequences(sequences, maxlen=100)
sentiment =  df['v1'].values
print('Shape of review tensor:', review_pad.shape)
print('Shape of sentiment tensor:', sentiment.shape)

# split the data into a training set and a validation set
indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
sentiment = sentiment[indices]
num_validation_samples = int(VALIDATION_SPLIT * review_pad.shape[0])

X_train_pad = review_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
X_test_pad = review_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

Found 7782 unique tokens.
Shape of review tensor: (5572, 100)
Shape of sentiment tensor: (5572,)


In [61]:
print('Shape of X_train_pad tensor:', X_train_pad.shape)
print('Shape of y_train tensor:', y_train.shape)

print('Shape of X_test_pad tensor:', X_test_pad.shape)
print('Shape of y_test tensor:', y_test.shape)

Shape of X_train_pad tensor: (4458, 100)
Shape of y_train tensor: (4458,)
Shape of X_test_pad tensor: (1114, 100)
Shape of y_test tensor: (1114,)


In [62]:
EMBEDDING_DIM =100
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [63]:
print(num_words)

7783


In [64]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.initializers import Constant

# define model
model = Sequential()
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=100,
                            trainable=False)

model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the model
model.fit(X_train_pad, y_train, batch_size=128, epochs=25, validation_data=(X_test_pad, y_test), verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          778300    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 96, 128)           64128     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 48, 128)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6144)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6145      
Total params: 848,573
Trainable params: 70,273
Non-trainable params: 778,300
_________________________________________________________________
None
Train on 4458 samples, validate on 1114 samples
Epoch 1/25
 - 2s - loss: 0.4547 - acc: 0.8437 - val_loss: 0.3330 - val_acc: 0.

In [65]:
# evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test, batch_size=128)
print('Accuracy: %f' % (accuracy*100))

1114/1114 [==============================] - 0s 131us/step
Accuracy: 89.048474


In [66]:
#Let us test some  samples
# load the dataset but only keep the top n words, zero the rest

test_sample_1 = "This movie is fantastic! I really like it because it is so good!"
test_sample_2 = "Good movie!"
test_sample_3 = "Maybe I like this movie."
test_sample_4 = "Not to my taste, will skip and watch another movie"
test_sample_5 = "if you like action, then this movie might be good for you."
test_sample_6 = "Bad movie!"
test_sample_7 = "Not a good movie!"
test_sample_8 = "This movie really sucks! Can I get my money back please?"
test_samples = [test_sample_1, test_sample_2, test_sample_3, test_sample_4, test_sample_5, test_sample_6, test_sample_7, test_sample_8]

test_samples_tokens = tokenizer_obj.texts_to_sequences(test_samples)
test_samples_tokens_pad = pad_sequences(test_samples_tokens, maxlen=3
                                       )

#predict
model.predict(x=test_samples_tokens_pad)

ValueError: Error when checking input: expected embedding_2_input to have shape (100,) but got array with shape (3,)

In [67]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

# define model
model = Sequential()
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=100,
                            trainable=False)
model.add(embedding_layer)
model.add(GRU(units=32,  dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Summary of the built model...')
print(model.summary())

Summary of the built model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          778300    
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 791,101
Trainable params: 12,801
Non-trainable params: 778,300
_________________________________________________________________
None


In [68]:
print('Train...')

model.fit(X_train_pad, y_train, batch_size=128, epochs=25, validation_data=(X_test_pad, y_test), verbose=2)

Train...
Train on 4458 samples, validate on 1114 samples
Epoch 1/25
 - 4s - loss: 0.4999 - acc: 0.8587 - val_loss: 0.4605 - val_acc: 0.8627
Epoch 2/25
 - 3s - loss: 0.4454 - acc: 0.8668 - val_loss: 0.4427 - val_acc: 0.8627
Epoch 3/25
 - 3s - loss: 0.4288 - acc: 0.8668 - val_loss: 0.4196 - val_acc: 0.8627
Epoch 4/25
 - 3s - loss: 0.3939 - acc: 0.8668 - val_loss: 0.3551 - val_acc: 0.8627
Epoch 5/25
 - 3s - loss: 0.3500 - acc: 0.8652 - val_loss: 0.3258 - val_acc: 0.8627
Epoch 6/25
 - 3s - loss: 0.3419 - acc: 0.8620 - val_loss: 0.3252 - val_acc: 0.8627
Epoch 7/25
 - 3s - loss: 0.3330 - acc: 0.8652 - val_loss: 0.3169 - val_acc: 0.8627
Epoch 8/25
 - 3s - loss: 0.3315 - acc: 0.8661 - val_loss: 0.3139 - val_acc: 0.8627
Epoch 9/25
 - 3s - loss: 0.3272 - acc: 0.8665 - val_loss: 0.3087 - val_acc: 0.8627
Epoch 10/25
 - 3s - loss: 0.3233 - acc: 0.8659 - val_loss: 0.3051 - val_acc: 0.8627
Epoch 11/25
 - 3s - loss: 0.3194 - acc: 0.8641 - val_loss: 0.3054 - val_acc: 0.8627
Epoch 12/25
 - 3s - loss: 0.

In [69]:
print('Testing...')
score, acc = model.evaluate(X_test_pad, y_test, batch_size=128)

print('Test score:', score)
print('Test accuracy:', acc)

print("Accuracy: {0:.2%}".format(acc))

Testing...
1114/1114 [==============================] - 0s 173us/step
Test score: 0.2910021193579669
Test accuracy: 0.8653500905156778
Accuracy: 86.54%
